In [ ]:
# Actividad 4: Clasificación multiclase con Scikit-learn y TensorFlow sobre dataset diamonds
# Carmen De Los Ángeles Camacho Tejada - 16/02/2025
# Ciencia De Datos - UNIR
# Profesor Alan Sastre - Módulo 2

# Importamos las librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Cargamos el dataset 'tips' desde seaborn
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv'
data = pd.read_csv(url)

# Mostramos las primeras filas del dataset
data.head()

*_ANÁLISIS DE ESTADÍSTICAS DESCRIPTIVAS_*

In [ ]:
# Información general del dataset
data.info()

In [ ]:
# Estadísticas de las variables numéricas del dataset
data.describe()

In [ ]:
# Descripción de las variables categóricas del dataset
data.describe(include=['object'])

In [ ]:
# Verificación de valores nulos
data.isnull().sum()

In [ ]:
# Distribución de la variable a predecir: 'cut'
data['cut'].value_counts()

In [ ]:
# Distribución de las otras variables categóricas
columnas_category = ['color', 'clarity']
for col in columnas_category:
    print(data[col].value_counts())

In [ ]:
# Para mejor entendimiento de la columna <cut>, se realiza un gráfico de barras
plt.figure(figsize=(8,5))
sns.set_style("whitegrid")
sns.countplot(x='cut', data=data, color="chocolate", edgecolor="black")
plt.title('Distribución de la variable Cut', fontsize=16, fontweight="bold", color="peru")
plt.show()


*_LIMPIEZA DE DATOS_*

1. Revisar valores nulos.
2. Detectar y tratar valores atípicos (outliers).
3. Revisar y convertir tipos de datos si es necesario.
4. Eliminar columnas irrelevantes.

Dado que en la anterior actividad, no llegué a tratar valores nulos ni duplicados, he decidido en esta, crear algunos y corregirlos posteriormente para practicar este paso (en la vida real, las bases de datos están llenas de estos).

In [ ]:
# VALORES NULOS
# Introducir valores nulos en algunas filas:
# Cada 50, se añadirá un valor nulo en la columna 'carat'
# Cada 75, se añadirá un valor nulo en la columna a predecir 'cut'
# Cada 100, se añadirá un valor nulo en la columna 'price'

data.loc[::50, 'carat'] = np.nan
data.loc[::75, 'cut'] = np.nan
data.loc[::100, 'price'] = np.nan

In [ ]:
# Revisar valores nulos
# Como se añadieron antes valores nulos, ahora SI deben aparecer aquí, diferenciándose de cuando se hizo más arriba
data.isnull().sum()

In [ ]:
# Rellenar valores nulos con distintos procedimientos vistos en clase:
# 'carat' -> la media
# 'cut' -> la moda
# 'price' -> la mediana

from sklearn.impute import SimpleImputer

# Se crea un imputador para cada columna con la estrategia correspondiente
imputer_carat = SimpleImputer(strategy='mean')
imputer_cut = SimpleImputer(strategy='most_frequent')
imputer_price = SimpleImputer(strategy='median')

# Se aplica el imputador a cada columna
data['carat'] = imputer_carat.fit_transform(data[['carat']]).flatten()
data['cut'] = imputer_cut.fit_transform(data[['cut']]).flatten()
data['price'] = imputer_price.fit_transform(data[['price']]).flatten()


In [ ]:
# Revisamos de nuevo los valores nulos
data.isnull().sum()

In [ ]:
# OUTLIERS
# Detectar y tratar valores atípicos:
# Se puede detectar estos usando diagramas de caja con las columnas numéricas
columnas_numerical = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
plt.figure(figsize=(10,6))
sns.set_style("white")
sns.boxplot(data=data[columnas_numerical], palette="viridis")
plt.title("Detección de valores atípicos", fontsize=16, fontweight='bold', color="lightblue")
plt.show()

In [ ]:
# Para tratar los outliers, se usará el valor inter cuartil, IQR
def detectar_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    return df[(df[column] < limite_inferior) | (df[column] > limite_superior)]

In [ ]:
for columna in columnas_numerical:
    outliers = detectar_outliers(data, columna)
    print(f"Outliers detectados en {columna}: {len(outliers)}")
    # Eliminar los outliers del dataset
    data = data[~data.index.isin(outliers.index)]


In [ ]:
# Visualizar con un boxplot para confirmar que no hay valores atípicos
plt.figure(figsize=(10,6))
sns.set_style("white")
sns.boxplot(data=data[columnas_numerical], palette="viridis")
plt.title('Boxplot de la columna depth después de eliminar outliers', fontsize=16, fontweight='bold', color="indianred")
plt.show()

for columna in columnas_numerical:
   outliers = detectar_outliers(data, columna)
   print(f"Outliers detectados en {columna}: {len(outliers)}")

Vemos que, aunque se han reducido en gran medida los valores atípicos, aún no se han eliminado por completo. Esto puede ser normal dependiendo del contexto y el tipo de datos. Por ejemplo, los outliers pueden representar diamantes muy caros o muy grandes. Por ahora, he decidido no eliminarlos porque podría distorsionar el análisis.

Más adelante, cuando se realize la preparación de datos, se probará el método de RobustScaler. Esta es una técnica de escalado que ayuda a reducir el impacto de los outliers en los datos. A diferencia de otros escaladores como StandardScaler (que usa la media y la desviación estándar), RobustScaler usa la mediana y el rango intercuartílico (IQR), lo que lo hace menos sensible a valores extremos. Como en la anterior actividad utilizé el StandardScaler, en esta probaré esta técnica para ver su funcionamiento y entender su diferencia.

In [ ]:
# TIPOS DE DATOS:
# Revisar y convertir tipos de datos si es necesario
data.dtypes

In [ ]:
# Voy a corregir las columnas 'cut', 'color', 'clarity' cambiándolas a tipo categoría
data = data.assign(
    cut=data['cut'].astype('category'),
    color=data['color'].astype('category'),
    clarity=data['clarity'].astype('category')
)
data.dtypes # Comprobación del arreglo

In [ ]:
# COLUMNAS IRRELEVANTES:
# Eliminar columnas irrelevantes
# En este dataset, la columna 'x', 'y', 'z'  se pueden considerar redundantes
data.drop(columns=['x', 'y', 'z'], inplace=True)

*_ANÁLISIS EXPLORATORIO DE DATOS (EDA)_*

- EDA Univariante: Objetivo -> Entender la distribución de cada variable por separado.
- EDA Bivariante: Objetivo -> Explorar la relación entre dos variables.
- EDA Multivariante: Objetivo -> Entender la interacción entre más de dos variables.


In [ ]:
# EDA UNIVARIANTE:
# Se realizan histograma de las columnas numéricas
variables_numeric = ['carat', 'depth', 'table', 'price']

for col in variables_numeric:
    plt.figure(figsize=(8, 4))
    sns.set_style("white")
    sns.histplot(data[col], kde=True, bins=30, color="olivedrab", edgecolor="black")
    plt.title(f'Distribución de {col}', fontsize=16, fontweight="bold", color="y")
    plt.xlabel(col, fontsize=16, fontweight="bold", color="darkseagreen")
    plt.ylabel('Frecuencia', fontsize=16, fontweight="bold", color="darkseagreen")
    plt.show()

In [ ]:
# Estadísticas descriptivas para variables numéricas
data[variables_numeric].describe()

In [ ]:
# Análisis de variables categóricas
# Se realizan histogramas de las variables categóricas
variables_categoric = ['cut', 'color', 'clarity']

for col in variables_categoric:
    plt.figure(figsize=(8, 4))
    sns.countplot(x=data[col], order=data[col].value_counts().index, color="firebrick", edgecolor="black")
    plt.title(f'Frecuencia de {col}', fontsize=16, fontweight="bold", color="sienna")
    plt.xlabel(col, fontsize=16, fontweight="bold", color="indianred")
    plt.ylabel('Frecuencia', fontsize=16, fontweight="bold", color="indianred")
    plt.show()



In [ ]:
# Frecuencia de las variables categóricas
for col in variables_categoric:
    print(f"\nDistribución de {col}:")
    print(data[col].value_counts())

En estas semanas también hemos visto la simetría o la asimetría de una distribución. Esta, medida con .skew(), forma parte del EDA univariante. Se utiliza principalmente para entender la distribución de las variables numéricas y evaluar si están sesgadas hacia la derecha (asimetría positiva) o hacia la izquierda (asimetría negativa).

In [ ]:
# Simetría de las variables numéricas
data[variables_numeric].skew()


Las variables carat, table, y price presentan asimetría a la derecha (sesgo positivo), lo que significa que la mayoría de los datos están concentrados en el lado inferior de su rango. Algunos valores son más altos y arrastran la cola de la distribución hacia la derecha.
En cambio, Depth tiene una ligera asimetría a la izquierda (sesgo negativo), lo que indica que la mayoría de los valores son relativamente altos, pero con algunos valores más bajos.

In [ ]:
#EDA BIVARIANTE:
# Diagrama de dispersión entre carat y price
sns.set_style("white")
sns.scatterplot(data=data, x='carat', y='price', color="palevioletred", edgecolor="black")
plt.title('Relación entre Carat y Price', fontsize=16, fontweight="bold", color="violet")
plt.xlabel('Carat', fontsize=16, fontweight="bold", color="rosybrown")
plt.ylabel('Price', fontsize=16, fontweight="bold", color="rosybrown")
plt.show()

In [ ]:
# Diagrama de dispersión entre depth y price
sns.set_style("white")
sns.scatterplot(data=data, x='depth', y='price', color="goldenrod", edgecolor="black" )
plt.title('Relación entre Depth y Price', fontsize=16, fontweight="bold", color="tan")
plt.xlabel('Depth', fontsize=16, fontweight="bold", color="moccasin")
plt.ylabel('Price', fontsize=16, fontweight="bold", color="moccasin")
plt.show()

En el primer caso, vemos como Price y Carat tienen una relación muy alta, a mayor tamaño, mayor será el precio. Sin embargo, no podemos decir lo mismo en la otra comparación con la profundidad.

In [ ]:
# Boxplot de price según cut
sns.set_style("white")
sns.boxplot(data=data, x='cut', y='price', color="lightcoral")
plt.title('Distribución de Price por Cut',  fontsize=16, fontweight="bold", color="red")
plt.xlabel('Cut', fontsize=16, fontweight="bold", color="darkred")
plt.ylabel('Price', fontsize=16, fontweight="bold", color="darkred")
plt.show()

In [ ]:
# Boxplot de carat según cut
sns.set_style("white")
sns.boxplot(data=data, x='cut', y='carat',color="lightcyan")
plt.title('Distribución de Carat por Cut',  fontsize=16, fontweight="bold", color="turquoise")
plt.xlabel('Cut', fontsize=16, fontweight="bold", color="skyblue")
plt.ylabel('Carat', fontsize=16, fontweight="bold", color="skyblue")
plt.show()

Los boxplots muestran las diferencias en la mediana, los cuartiles y los valores atípicos entre las categorías de cut. Por ejemplo, se puede ver que el cut más alto tiende a tener diamantes de mayor precio y mayor tamaño.

In [ ]:
# Crear una tabla de contingencia entre dos variables categóricas
contingency_table = data.pivot_table(index=['cut', 'color'], columns='clarity', aggfunc='size', fill_value=0, observed=False)
contingency_table


Una tabla de contingencia es una forma de visualizar cómo se distribuyen las categorías de dos o más variables categóricas. Te permite ver cuántas veces aparecen combinaciones específicas de categorías.

In [ ]:
# Gráfico de barras apiladas comparando 'cut' y 'color'
sns.set_style('whitegrid')
pd.crosstab(data['cut'], data['color']).plot(kind='bar', stacked=True, cmap='viridis')
plt.title('Distribución de Cut y Color', fontsize=16, fontweight="bold", color="mediumspringgreen")
plt.xlabel('Cut', fontsize=16, fontweight="bold", color="teal")
plt.ylabel('Frecuencia', fontsize=16, fontweight="bold", color="teal")
plt.show()

# Gráfico de barras apiladas comparando 'cut' y 'clarity'
sns.set_style('whitegrid')
pd.crosstab(data['cut'], data['clarity']).plot(kind='bar', stacked=True, cmap='flare')
plt.title('Distribución de Cut y Clarity', fontsize=16, fontweight="bold", color="plum")
plt.xlabel('Cut', fontsize=16, fontweight="bold", color="palevioletred")
plt.ylabel('Frecuencia', fontsize=16, fontweight="bold", color="palevioletred")
plt.show()


Un gráfico de barras apiladas es útil para ver la distribución de las categorías en relación con otras variables categóricas. Esto te permite comparar cómo se distribuyen las categorías dentro de otra categoría.

In [ ]:
# Calculando la matriz de correlación entre las variables numéricas
corr_matrix = data[['carat', 'depth', 'table', 'price']].corr()

# Mapa de calor de la matriz de correlación
plt.figure(figsize=(8, 6))
sns.set_style("white")
sns.heatmap(corr_matrix, annot=True, cmap='rocket', fmt='.2f', linewidths=0.5)
plt.title('Mapa de Calor de Correlación entre Variables Numéricas', fontsize=16, fontweight="bold", color="red")
plt.show()


In [ ]:
#Se puede ver la relación de las variables categóricas utilizando la matriz de correlación transformándolas en variables numéricas mediante técnicas Label Encoding.
from sklearn.preprocessing import LabelEncoder

# Codificar variables categóricas en valores numéricos
le = LabelEncoder()
data_codify = data.copy()
data_codify['cut'] = le.fit_transform(data['cut'])
data_codify['color'] = le.fit_transform(data['color'])
data_codify['clarity'] = le.fit_transform(data['clarity'])

# Matriz de correlación
corr_matrix_categor = data_codify[['cut', 'color', 'clarity']].corr()

# Mapa de calor de la correlación
sns.set_style('whitegrid')
sns.heatmap(corr_matrix_categor, annot=True, cmap='viridis_r', fmt='.2f')
plt.title('Mapa de Calor de Correlación entre Variables Categóricas', fontsize=16, fontweight="bold", color="deepskyblue")
plt.show()


In [ ]:
#EDA MULTIVARIANTE:
# Pairplot de las variables numéricas (carat, depth, table, price)
plt.figure(figsize=(8, 6))
sns.set_style("white")
sns.pairplot(data[['carat', 'depth', 'table', 'price']], diag_kind='kde')
plt.suptitle('Gráfico de Pares: Relación entre variables', y=1.02, fontsize=16, fontweight="bold", color="royalblue")
plt.show()


Un gráfico de pares o pairplot muestra todas las combinaciones de pares de variables, lo que te permite ver patrones, correlaciones y posibles agrupamientos entre ellas.

In [ ]:
# Crear un gráfico de violín para cada columna numérica en función de 'cut'
col_numeric = data.select_dtypes(include=['float64', 'int64']).columns
for col in col_numeric:
    plt.figure(figsize=(8, 6))
    sns.violinplot(data=data, hue='cut', y=col, palette='magma')
    plt.title(f'Distribución de {col} por Cut', fontsize=16, fontweight="bold", color="tomato")
    plt.legend(title='Cut', loc='center', bbox_to_anchor=(0.5, -0.05), ncol=2)
    plt.show()

Los gráficos de violín son útiles para comparar las distribuciones de una variable numérica en diferentes categorías. En nuestro caso, se puede comparar la distribución de variables como price, carat, etc., para cada categoría de cut. Los violines más anchos indican una mayor concentración de valores en esa área. Es útil para ver si las distribuciones de price o carat son similares o difieren dependiendo del cut.

Como en las clases también hemos mencionado la libreria plotly express para realizar gráficos interactivos, voy a trabajar un poco con ella.

In [ ]:
import plotly.express as px

#Gráfico de dispersión 3D para visualizar la relación entre tres variables numéricas: carat, depth y price. Además, usaremos el parámetro color para diferenciar los puntos según la variable categórica cut.

fig = px.scatter_3d(data,
                    x='carat',
                    y='depth',
                    z='price',
                    color='cut',
                    title="Gráfico 3D: Relación entre Carat, Depth y Price",
                    labels={"carat": "Carat", "depth": "Depth", "price": "Price"})


fig.show()


In [ ]:
fig = px.line(data, x="price", y="carat", color="cut", line_group="cut", title="Carat vs Price por Cut")
fig.show()


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Seleccionamos las variables numéricas
df_num = data[['carat', 'depth', 'table', 'price']]

# Estandarizamos las variables
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_num)

# Aplicamos PCA
pca = PCA(n_components=2)  # Reduciendo a 2 componentes principales
pca_components = pca.fit_transform(df_scaled)

# Creamos un DataFrame con los componentes principales
pca_df = pd.DataFrame(pca_components, columns=['PC1', 'PC2'])

# Gráfica con los componentes principales
plt.figure(figsize=(8, 6))
plt.scatter(pca_df['PC1'], pca_df['PC2'], alpha=0.5, cmap='flare', c=data['price'])
plt.title('PCA: Componentes Principales', fontsize=16, fontweight="bold", color="red")
plt.xlabel('Componente Principal 1', fontsize=16, fontweight="bold", color="tomato")
plt.ylabel('Componente Principal 2', fontsize=16, fontweight="bold", color="tomato")
plt.colorbar(label='Price')
plt.show()

# Ver la varianza explicada por los componentes
print(f"Varianza explicada por PC1: {pca.explained_variance_ratio_[0]:.4f}")
print(f"Varianza explicada por PC2: {pca.explained_variance_ratio_[1]:.4f}")
print(f"Varianza total explicada: {sum(pca.explained_variance_ratio_):.4f}")


PCA es útil cuando se tienen muchas variables numéricas y se desea reducir la dimensionalidad mientras se mantiene la mayor parte de la variabilidad en los datos. Esto nos puede ayudar a visualizarlos de forma más clara y también identificar patrones o agrupamientos. En el gráfico de PCA, se puede ver cómo los datos se agrupan en función de los componentes principales.

En este caso, vemos que los datos están agrupados en el centro, lo que puede sugerir una baja Varianza en los datos originales, los datos están muy correlacionados, o incluso la estandarización es insuficiente o incorrecta. Por este último motivo vamos a aplicar el método mencionado anteriormente, RobustScaler.


In [ ]:
from sklearn.preprocessing import RobustScaler

# Estandarizamos las variables con RobustScaler
scaler = RobustScaler()
df_robust_scaled = scaler.fit_transform(df_num)

# Aplicamos PCA
pca = PCA(n_components=2)  # Reduciendo a 2 componentes principales
pca_components = pca.fit_transform(df_robust_scaled)

# Crear un DataFrame con los componentes principales
pca_robust_df = pd.DataFrame(pca_components, columns=['PC1', 'PC2'])

# Graficar los componentes principales
plt.figure(figsize=(8, 6))
plt.scatter(pca_robust_df['PC1'], pca_robust_df['PC2'], alpha=0.5, cmap='viridis', c=data['price'])
plt.title('PCA con RobustScaler: Componentes Principales',fontsize=16, fontweight="bold", color="c")
plt.xlabel('Componente Principal 1', fontsize=16, fontweight="bold", color="dodgerblue")
plt.ylabel('Componente Principal 2', fontsize=16, fontweight="bold", color="dodgerblue")
plt.colorbar(label='Price')
plt.show()

# Ver la varianza explicada por los componentes
print(f"Varianza explicada por PC1: {pca.explained_variance_ratio_[0]:.4f}")
print(f"Varianza explicada por PC2: {pca.explained_variance_ratio_[1]:.4f}")
print(f"Varianza total explicada: {sum(pca.explained_variance_ratio_):.4f}")



Resultados:
La varianza total explicada es muy similar en ambos casos, lo que significa que, en términos generales, ambas escalas logran capturar prácticamente la misma cantidad de información sobre los datos. StandardScaler parece explicar mejor la varianza en el primer componente principal (PC1), con un 49.35% frente al 41.96% de RobustScaler. Esto indica que, con StandardScaler, el primer componente principal captura más de la variabilidad de los datos. Sin embargo, RobustScaler parece distribuir la varianza de manera más equilibrada entre PC1 (41.96%) y PC2 (37.45%).

Dado que nuestros datos tienen outliers o valores extremos, RobustScaler es una la mejor opción, ya que se basa en la mediana y el rango intercuartílico, haciéndola menos sensible a los outliers.

*_PREPARACIÓN DE DATOS PARA EL MODELADO_*

Resumen del proceso de preparación de datos:

+ Manejo de valores nulos → Imputación o eliminación. ✓
+ Codificación de variables categóricas → One-Hot o Label Encoding. ✓
+ Escalado de datos numéricos → Estandarización o normalización.
+ Eliminación de variables irrelevantes ✓
+ División en conjuntos de entrenamiento y prueba.

✓ Indica aquellos pasos ya realizados.



In [ ]:
# Revisamos la codificación con Label Encoding
data_codify.head()

In [ ]:
# Se añade nuevas columnas que aporten mayor información al dataset
# Precio por quilate: Puede ayudar a diferenciar calidades
data_codify['price_per_carat'] = data_codify['price'] / data_codify['carat']

# Relación depth con table: Puede indicar qué proporción del diamante está en la tabla, lo que influye en su brillo
data_codify['depth_table_ratio'] = data_codify['depth'] / data_codify['table']

# Relación entre color y clarity: Algunas combinaciones de color y claridad pueden influir en el cut.
data_codify['color_clarity'] = data_codify['color'].astype(str) + "_" + data_codify['clarity'].astype(str)

data_codify.head()

In [ ]:
# Se aplicar Label Encoding con la columna color_clarity_interaction
encoder = LabelEncoder()
data_codify['color_clarity'] = encoder.fit_transform(data_codify['color_clarity'])

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

# Se separa la variable (y) y las variables predictoras (X)
X = data_codify.drop(columns=['cut'])
y = data_codify['cut']

# Se aplica RobustScaler solo a las variables numéricas
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# Se convierte a DataFrame para facilitar su uso
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Se imprime para comprobarlo
X_scaled.head()

In [ ]:
# Se divide los datos en entrenamiento y prueba (80% train - 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Se verifica el tamaño de ambos
print(f"Tamaño de X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"Tamaño de y_train: {y_train.shape}, y_test: {y_test.shape}")


*_MODELADO_*

En mi caso, he decidido usar regresión Logística (Baseline simple), Random Forest (Modelo basado en árboles), XGBoost (Modelo avanzado de boosting) y una red neuronal con TensorFlow. Al final se comparará el rendimiento de todos y se tomará una decision sobre cuál será mejor emplear.

Pasos para el modelado:

- Importar librerías necesarias.
- Optimización de hiperparámetros
- Crear y entrenar el modelo de regresión lineal.
- Realizar predicciones con el conjunto de prueba.
- Evaluar el modelo usando métricas adecuadas.


In [ ]:
# Modelos con Scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Lista de modelos con sus respectivos nombres
modelos = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42),
    "SVC": SVC(random_state=42),
    "K-Neighbors": KNeighborsClassifier()
}

# Diccionario para almacenar las métricas
resultados_sklearn = {}

# Entrenamiento y evaluación de los modelos mencionados antes
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)  # Se entrena el modelo
    y_pred = modelo.predict(X_test)  # Se predicen los resultados
    acc = accuracy_score(y_test, y_pred)  # Se calcula la precisión
    precision = precision_score(y_test, y_pred, average='weighted')  # Promedio ponderado
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    # Validación cruzada con 5 folds
    cross_val_acc = cross_val_score(modelo, X_train, y_train, cv=5, scoring='accuracy').mean()
    # Se guardan los resultados
    resultados_sklearn[nombre] = {
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Cross-Val Accuracy": cross_val_acc
    }

# Mostrar resultados
print("\n Comparación de Modelos Scikit-Learn:")
for nombre, metricas in resultados_sklearn.items():
    print(f"\n Modelo --> {nombre}:")
    for metrica, valor in metricas.items():
        print(f" Metrica --> {metrica}: Valor --> {valor:.4f}")


_*CONCLUSIONES*_

1. La regresión logística muestra una precisión moderada, pero su F1-Score es relativamente bajo, lo que sugiere un desequilibrio entre las métricas de precisión y recall. El modelo tiene un desempeño aceptable, pero no es el mejor en términos de precisión y F1-Score.

2. El Random Forest muestra un buen rendimiento en todas las métricas, especialmente en términos de precisión y recall, con una F1-Score decente. Este modelo parece ser una de las opciones más equilibradas y tiene un buen desempeño en validación cruzada. Es un modelo robusto para clasificación.

3. El XGBoost se destaca como el mejor modelo en todas las métricas clave, superando a los demás en precisión, recall y F1-Score. También tiene un excelente rendimiento en validación cruzada. Es el modelo más efectivo y parece ser la mejor opción si se busca maximizar el rendimiento.

4. SVC (Support Vector Classifier) presenta un rendimiento sólido, con un equilibrio entre precisión y recall. Aunque no supera a XGBoost o Random Forest, sigue siendo una opción competitiva con una buena precisión y rendimiento general.

5. K-Neighbors tiene un rendimiento moderado, similar al de la regresión logística, aunque con una precisión algo mayor. Sin embargo, su F1-Score sigue siendo inferior al de Random Forest y XGBoost, lo que sugiere que no es tan efectivo en términos de balance entre precisión y recall.

En resumen: XGBoost posee mejor rendimiento general con la mayor precisión, recall, F1-Score y cross-validation. Le acompaña Random Forest, con un buen rendimiento aunque ligeramente inferior y posee un desempeño consistente en todas las métricas. El SVC tiene un rendimiento competitivo, pero no tan fuerte como los dos anteriores, ofreciendo un buen equilibrio en precisión y recall. Finalmente, Logistic Regression y K-Neighbors tienen resultados relativamente cercanos, pero no alcanzan el rendimiento de los modelos más avanzados como XGBoost y Random Forest.

Si se busca el mejor desempeño posible, XGBoost es la opción más adecuada. Por otra parte, Random Forest es una excelente alternativa si se busca un modelo robusto y equilibrado. También, SVC puede ser útil si se necesita un modelo que balancee precisión y recall sin requerir demasiado ajuste. Por último, Logistic Regression y K-Neighbors son modelos más simples y adecuados si el objetivo es obtener resultados rápidos o si los recursos son limitados.
